In [82]:
import pandas as pd
import numpy as np
import json
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from lxml import html

In [87]:
with open('auth.json', 'r') as f:
    auth = json.load(f)

url = "https://s20.si0.kabu.co.jp/members/"
driver = webdriver.Chrome(executable_path=r'C:\Users\KOJ\PycharmProjects\untitled\GitMaster\auto_Investment\chromedriver')
driver.get(url)
driver.find_element_by_name('SsLogonUser').send_keys(auth['id'])
driver.find_element_by_name('SsLogonPassword').send_keys(auth['APIPassword'])
driver.find_element_by_id('image1').click()

In [88]:
s = requests.Session()
for cookie in driver.get_cookies():
    s.cookies.set(cookie['name'], cookie['value'])

In [89]:
ETF_Link = pd.read_csv('ETF_link.csv', encoding='utf-8')

In [90]:
ETF_Link = ETF_Link.values

In [91]:
etfNum = ETF_Link[0][0].split('.')[-2].split('=')[-1]
timeSeriesLink = "https://s20.si0.kabu.co.jp/ap/PC/InvInfo/Market/StockDetail/Default?Symbol=" + etfNum + "&Exchange=TSE&Category=PRICE_HISTORY"
result = s.get(timeSeriesLink).text
soup = BeautifulSoup(result, "html.parser")
table = soup.find_all("table", class_="tb3")
df = pd.read_html(str(table), header=0)[0]


In [92]:
df = df.drop("前日比", axis=1)
df = df.rename(columns={'基準日':'Date', '始値':'Open', '高値':'High', '安値':'Low', '終値':'Close', '出来高':'Volume'})

In [93]:
def fix_date(row):
    return row.split('(')[0]
df['Date'] = df['Date'].apply(fix_date)

In [94]:
df = df.replace('--', np.nan)
df = df.dropna(axis=0)

In [95]:
print(df)

          Date   Open   High    Low  Close Volume
0   2021/03/05  13130  13260  13130  13260     69
1   2021/03/04  13190  13190  13130  13190     21
2   2021/03/03  13120  13190  13120  13190     11
3   2021/03/02  13170  13170  13170  13170     10
4   2021/03/01  13060  13120  13060  13120      6
5   2021/02/26  13210  13210  13060  13060     21
6   2021/02/25  13100  13200  13100  13200     88
7   2021/02/24  13120  13130  13060  13100    200
8   2021/02/22  13090  13160  13090  13140    112
9   2021/02/19  13200  13230  13130  13130     20
10  2021/02/18  13290  13290  13200  13200     33
11  2021/02/17  13230  13290  13210  13290     79
12  2021/02/16  13190  13230  13190  13230    327
13  2021/02/15  13120  13190  13120  13190     62
14  2021/02/12  13110  13150  13110  13110     29
15  2021/02/10  13170  13170  13110  13110     62
16  2021/02/09  13130  13190  13110  13170     18
17  2021/02/08  13040  13140  13040  13140     14
18  2021/02/05  13180  13180  13060  13160     40


/ap/PC/InvInfo/Market/StockDetail/ListCsv?SymbolCode=1349&SymbolName=%EF%BC%A1%EF%BC%A2%EF%BC%A6%E6%B1%8E%E3%82%A2%E3%82%B8%E3%82%A2%E5%82%B5%E5%88%B8%E3%82%A4%E3%83%B3%E3%83%87%E3%83%83%E3%82%AF%E3%82%B9%E3%83%BB%E3%83%95%E3%82%A1%E3%83%B3%E3%83%89&exchange=TSE&exchangeName=%E6%9D%B1%E4%BA%AC%20ETF%2FETN
